In [1]:
import sys
sys.path.append('/home/wangs/rs/lib')
import ff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from multiprocessing import Pool
from tqdm import tqdm

In [2]:
from scipy.stats import pearsonr,spearmanr

In [3]:
from gplearn.genetic import SymbolicRegressor,SymbolicTransformer
from gplearn import fitness
from gplearn.functions import make_function
from gplearn.fitness import make_fitness

In [4]:
post=ff.read('post')
filter0=ff.filter0
close=ff.read('close')*post*filter0
open_=ff.read('open')*post*filter0
high=ff.read('high')*post*filter0
low=ff.read('low')*post*filter0
vol=ff.read('vol')*filter0
amount=ff.read('amount')*post*filter0

In [5]:
close

,20100104,20100105,20100106,20100107,20100108,20100111,20100112,20100113,20100114,20100115,...,20240517,20240520,20240521,20240522,20240523,20240524,20240527,20240528,20240529,20240530
000001.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1332.86246,1328.19394,1348.03515,1349.20228,1330.52820,1320.02403,1343.36663,1330.52820,1314.18838,1297.84856
000002.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1635.33600,1668.04272,1669.85976,1708.01760,1737.09024,1633.51896,1611.71448,1548.11808,1559.02032,1502.69208
000004.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,46.24832,48.52416,47.34560,47.14240,46.28896,44.74464,44.37888,43.40352,43.40352,42.63136
000005.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000006.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,171.67680,166.11320,166.51060,168.89500,164.52360,160.15220,158.16520,153.79380,155.78080,150.21720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873726.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873806.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873833.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
920002.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
ff.cal_returns??

Signature: ff.cal_returns(returns)
Docstring: <no docstring>
Source:   
def cal_returns(returns):
    result={}
    cumsum=returns.cumsum()
    result['年化收益率']=cumsum.iloc[-1]/len(cumsum)*250
    result['年化波动率']=(returns).std()*(250**0.5)
    result['夏普率']=result['年化收益率']/result['年化波动率']
    result['最大回撤']=cal_downdraw((returns).cumsum())
    result['收益回撤比']=-result['年化收益率']/result['最大回撤']
    result['胜率']=round(len(returns[returns>0])/len(returns),3)
    result['盈亏比']=-returns[returns>0].mean()/returns[returns<=0].mean()
    return result
File:      ~/rs/lib/ff.py
Type:      function

In [7]:
mkv=(ff.read('mkv','factor')*filter0).loc[close.index,'20200101':'20210101']
mkv_s=mkv.values.reshape(-1)

In [8]:
inds = pd.read_pickle('/mydata2/wangs/data/dict_ind_matrix_sw1.pk')

In [9]:
mkt20=close.shift(-20,axis=1)/close - 1

In [ ]:
stock_test=pd.DataFrame({
                'accelerated_turnover_rank_RC':pd.DataFrame(ff.read('accelerated_turnover_rank_RC').loc[ff.read('synergy').index,'20200101':'20210101'],index=open_.index).values.reshape(-1),
                         'CSK_XYY_UP_DOWN_120D_RC':pd.DataFrame(ff.read('CSK_XYY_UP_DOWN_120D_RC').loc[ff.read('synergy').index,'20200101':'20210101'],index=open_.index).values.reshape(-1),
                         'high_fre_vol_RC':pd.DataFrame(ff.read('high_fre_vol_RC').loc[ff.read('synergy').index,'20200101':'20210101'],index=open_.index).values.reshape(-1),
                         'high_fre_diff_vol_RC':pd.DataFrame(ff.read('high_fre_diff_vol_RC').loc[ff.read('synergy').index,'20200101':'20210101'],index=open_.index).values.reshape(-1),
                         'high_fre_absdiff_vol_RC':pd.DataFrame(ff.read('high_fre_absdiff_vol_RC').loc[ff.read('synergy').index,'20200101':'20210101'],index=open_.index).values.reshape(-1),
                         'peak_count_vol_RC':pd.DataFrame(ff.read('peak_count_vol_RC').loc[ff.read('synergy').index,'20200101':'20210101'],index=open_.index).values.reshape(-1),
                         'overnightsmart20_RC':pd.DataFrame(ff.read('overnightsmart20_RC').loc[ff.read('synergy').index,'20200101':'20210101'],index=open_.index).values.reshape(-1),
                         'CTR_RC':pd.DataFrame(ff.read('CTR_RC').loc[ff.read('synergy').index,'20200101':'20210101'],index=open_.index).values.reshape(-1),
                         'jumpCTR_RC':pd.DataFrame(ff.read('jumpCTR_RC').loc[ff.read('synergy').index,'20200101':'20210101'],index=open_.index).values.reshape(-1),
                         'turnover_rate_proportion_l':pd.DataFrame(ff.read('turnover_rate_proportion_l').loc[ff.read('synergy').index,'20200101':'20210101'],index=open_.index).values.reshape(-1),
                         'synergy':pd.DataFrame(ff.read('synergy').loc[ff.read('synergy').index,'20200101':'20210101'],index=open_.index).values.reshape(-1),
                        })
test_code=[code for code in close.index for i in close.loc[:,'20200101':'20210101'].columns]
test_date=close.loc[:,'20200101':'20210101'].columns.to_list() *len(close.index)
stock_test['code']=test_code
stock_test['date']=test_date
stock_test['1'],stock_test['5'],stock_test['10'],stock_test['20'],stock_test['40'],stock_test['60']=1,5,10,20,40,60

In [ ]:
test_mkt=mkt20.loc[:,'20200101':'20210101'].values.reshape(-1)

In [ ]:
stock_test

In [ ]:
unique_code=close.index.to_list()  # list
unique_date=close.loc[:,'20200101':'20210101'].columns.to_list()

In [ ]:
fields=['accelerated_turnover_rank_RC','CSK_XYY_UP_DOWN_120D_RC','high_fre_vol_RC','high_fre_diff_vol_RC','high_fre_absdiff_vol_RC','peak_count_vol_RC','overnightsmart20_RC','CTR_RC','jumpCTR_RC','turnover_rate_proportion_l','synergy','1','5','20','40','60']

In [ ]:
# 自带算子
init_function = ['add','sub','mul','div','sqrt','log','inv','abs','neg']

In [ ]:
# 自定义算子
def _Add(data1,data2):
    if len(np.unique(data1))<=1 or len(np.unique(data2))<=1:
        return np.zeros(len(data1))
    else:
        return np.add(data1,data2)
    
def _Sub(data1,data2):
    if len(np.unique(data1))<=1 or len(np.unique(data2))<=1:
        return np.zeros(len(data1))
    else:
        return np.subtract(data1,data2)
    
def _Mul(data1,data2):
    if len(np.unique(data1))<=1 or len(np.unique(data2))<=1:
        return np.zeros(len(data1))
    else:
        return np.multiply(data1,data2)
    
def _Div(data1,data2):
    def protect_division(data1,data2):
        with np.errstate(divide='ignore',invalid='ignore'):
            return np.where(np.abs(data2)>0.001,np.divide(data1,data2),0.)
    if len(np.unique(data1))<=1 or len(np.unique(data2))<=1:
        return np.zeros(len(data1))
    else:
        return protect_division(data1,data2)
    
def _Sqrt(data):
    if len(np.unique(data))<=1:
        return np.zeros(len(data))
    else:
        return np.sqrt(np.abs(data))
    
def _Log(data):
    def protect_log(data):
        with np.errstate(divide='ignore',invalid='ignore'):
            return np.where(np.abs(data) > 0.001, np.log(np.abs(data)), 0.)
    if len(np.unique(data))<=1:
        return np.zeros(len(data))
    else:
        return protect_log(data)
    
def _Inv(data):
    data=np.array(data)
    def protect_inv(data):
        with np.errstate(divide='ignore',invalid='ignore'):
            return np.where(np.abs(data) > 0.001, 1./data, 0.)
    if len(np.unique(data))<=1:
        return np.zeros(len(data))
    else:
        return protect_inv(data)

def _ts_max(data,window): #历史rolling最大
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code')['data'].transform(lambda x:x.rolling(window).max())
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data))

def _ts_min(data,window): #历史rolling最小
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code')['data'].transform(lambda x:x.rolling(window).min())
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data))

def _ts_mid(data,window): #历史rolling中位数
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code')['data'].transform(lambda x:x.rolling(window).median())
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data))


def _ts_mean(data,window):  #历史rolling平均
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code')['data'].transform(lambda x:x.rolling(window).mean())
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data))


def _ts_std(data,window): #历史rolling std
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code')['data'].transform(lambda x:x.rolling(window).std())
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data))
    
    
def _ts_sum(data,window): #历史rolling求和
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code')['data'].transform(lambda x:x.rolling(window).mean())
        return np.nan_to_num(value.values*window)
    except:
        return np.zeros(len(data))
    
    
def _ts_product(data,window): #历史rolling求积
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':np.log(data)})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code')['data'].transform(lambda x:x.rolling(window).mean())
        return np.nan_to_num(np.exp(value.values*window))
    except:
        return np.zeros(len(data))

    
def _delay(data,window):  # 几天以前的因子值
    window=window[0]
    if window not in [1,5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code')['data'].transform(lambda x : x.shift(window))
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data))

    
def _delta(data,window):  # 因子值与几天前的因子值之差
    window=window[0]
    if window not in [1,5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code')['data'].transform(lambda x : x.shift(window))
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data))

    
def _rank(data):  # 因子在截面上的分位数
    if len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('date')['data'].transform(lambda s : s.rank()/s.count())
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data))
    
    
def _ts_rank(data,window):  # 因子在过去几天中的分位数
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code  
        value=df.groupby('code')['data'].transform(lambda x:x.rolling(window).rank()/window)
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data))
    
    
def _sigmoid(data):
    data=np.array(data)
    with np.errstate(over='ignore',under='ignore'):
        return 1/(1+np.exp(-1*data))
    
    
def _correlation(data1,data2,window):
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data1))<=2 or len(np.unique(data2))<=2:
        return np.zeros(len(data1))
    try:
        df=pd.DataFrame({'data1':data1,'data2':data2})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code').apply(lambda df : df.data1.rolling(window).corr(df.data2)) # 因为code排序是按顺序的所以可以用apply
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data1))
    
    
def _covariance(data1,data2,window):
    window=window[0]
    if window not in [5,10,20,40,60] or len(np.unique(data1))<=2 or len(np.unique(data2))<=2:
        return np.zeros(len(data1))
    try:
        df=pd.DataFrame({'data1':data1,'data2':data2})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code').apply(lambda df : df.data1.rolling(window).cov(df.data2)) # 因为code排序是按顺序的所以可以用apply
        return np.nan_to_num(value.values)
    except:
        return np.zeros(len(data1))
    
    
def _scale(data):
    if len(np.unique(data))<=2:
        return np.zeros(len(data))
    else:
        return np.divide(data,np.sum(np.abs(data)))
    
    
def _decay_linear(data,window):  # 因子过去几天的加权平均值，权数随时间往前线性递减
    window=window[0]    
    if  window not in [5,10,20,40,60] or len(np.unique(data))<=2:
        return np.zeros(len(data))
    try:
        w=np.arange(1,window+1)
        w=w/np.sum(w)
        df=pd.DataFrame({'data':data})
        df['date']=test_date
        df['code']=test_code
        value=df.groupby('code').data.transform(lambda x:x.rolling(window).apply(lambda s:np.dot(w,s.values)).values)
        return np.nan_to_num(value)
    except:
        return np.zeros(len(data))

In [ ]:
Add=make_function(function=_Add,name='Add',arity=2)
Sub=make_function(function=_Sub,name='Sub',arity=2)
Mul=make_function(function=_Mul,name='Mul',arity=2)
Div=make_function(function=_Div,name='Div',arity=2)
Log=make_function(function=_Log,name='Log',arity=1)
Sqrt=make_function(function=_Sqrt,name='Sqrt',arity=1)
Inv=make_function(function=_Inv,name='Inv',arity=1)
ts_max=make_function(function=_ts_max,name='ts_max',arity=2) 
ts_min=make_function(function=_ts_min,name='ts_min',arity=2) 
ts_mid=make_function(function=_ts_mid,name='ts_mid',arity=2) 
ts_mean=make_function(function=_ts_mean,name='ts_mean',arity=2) 
ts_std=make_function(function=_ts_std,name='ts_std',arity=2)
delay=make_function(function=_delay,name='delay',arity=2)
delta=make_function(function=_delta,name='delta',arity=2)
rank=make_function(function=_rank,name='rank',arity=1)
sigmoid=make_function(function=_sigmoid,name='sigmoid',arity=1)
correlation=make_function(function=_correlation,name='correlation',arity=3)
covariance=make_function(function=_covariance,name='covariance',arity=3)
scale=make_function(function=_scale,name='scale',arity=1)
ts_rank=make_function(function=_ts_rank,name='ts_rank',arity=2)
ts_sum=make_function(function=_ts_sum,name='ts_sum',arity=2)
ts_product=make_function(function=_ts_product,name='ts_product',arity=2)

In [ ]:
user_function=[ts_max,ts_min,ts_mid,ts_mean,ts_std,delay,delta,rank,correlation,ts_rank]
function_set=init_function+user_function

In [ ]:
import statsmodels.api as sm

def rankIC_metric(y,y_pred,w=None):
    
    def cal_rankIC(df):
        df.index=df.code
        # 去极值
        M=df.y_pred.mean()    #df.y_pred.median()
        M1=df.y_pred.std()    #(df.y_pred-M).abs().median()
        df.y_pred.loc[df.y_pred>M+3*M1]=M+3*M1    #5
        df.y_pred.loc[df.y_pred<M-3*M1]=M-3*M1
        # 行业市值中性化
        day=df.date.iloc[0]
        ind=inds[day].copy()
        ind['mkv']=df.loc[ind.index,'mkv']
        input_x=ind.dropna().astype(float)
        input_y=df.loc[input_x.index,'y_pred']
        y_true=df.loc[input_x.index,'y'].values
        if len(input_x)==0:
            return 0
        else:
            mod=sm.OLS(input_y,input_x)
            res=mod.fit()
            y_adj=res.resid
            #标准化
            y_adj=(y_adj-y_adj.mean())/y_adj.std()
        
            if len(np.unique(y_adj.values)) <=2:
                return 0
            else:
                #return y_adj.corr(input_xy.mkt_delay,method='spearman')
                return spearmanr(np.nan_to_num(y_adj.values),np.nan_to_num(y_true))[0]
            
    if len(np.unique(y_pred))<=2:
        return 0
    else:
        try:
            data=pd.DataFrame({'y_pred':y_pred,'y':y,'mkv':np.log(mkv_s),'code':test_code,'date':test_date})
            res=data.groupby('date').apply(cal_rankIC)
            return np.abs(np.nanmean(res))
        except:
            return 0

In [ ]:
rankIC=make_fitness(function=rankIC_metric,greater_is_better=True)

In [ ]:
test_gp=SymbolicTransformer(feature_names = fields,
                            function_set = function_set, #所有算子
                            generations = 3, #进化代数
                            population_size = 100, #种群规模
                            tournament_size = 10, #竞标赛规模
                            p_crossover=0.4,
                            p_subtree_mutation=0.05,
                            p_hoist_mutation=0.01,
                            p_point_mutation=0.03,
                            p_point_replace=0.35,
                            init_depth=(1,4),
                            const_range = None,
                            metric=rankIC,
                            parsimony_coefficient = 'auto',
                            low_memory=True,
                            verbose=2,
                            n_jobs = 6)

In [ ]:
test_gp.fit(np.nan_to_num(stock_test.loc[:,fields].values),np.nan_to_num(test_mkt))

In [ ]:
stock_all=pd.DataFrame({
    # 'open':open_.loc[:,'20200101':'20210101'].values.reshape(-1),
    #                      'close':close.loc[:,'20200101':'20210101'].values.reshape(-1),
    #                      'high':high.loc[:,'20200101':'20210101'].values.reshape(-1),
    #                      'low':low.loc[:,'20200101':'20210101'].values.reshape(-1),
    #                      'vol':vol.loc[:,'20200101':'20210101'].values.reshape(-1),
    #                      'amount':amount.loc[:,'20200101':'20210101'].values.reshape(-1),
                'accelerated_turnover_rank_RC':pd.DataFrame(ff.read('accelerated_turnover_rank_RC').loc[ff.read('synergy').index,'20200101':'20240415'],index=open_.index).values.reshape(-1),
                         'CSK_XYY_UP_DOWN_120D_RC':pd.DataFrame(ff.read('CSK_XYY_UP_DOWN_120D_RC').loc[ff.read('synergy').index,'20200101':'20240415'],index=open_.index).values.reshape(-1),
                         'high_fre_vol_RC':pd.DataFrame(ff.read('high_fre_vol_RC').loc[ff.read('synergy').index,'20200101':'20240415'],index=open_.index).values.reshape(-1),
                         'high_fre_diff_vol_RC':pd.DataFrame(ff.read('high_fre_diff_vol_RC').loc[ff.read('synergy').index,'20200101':'20240415'],index=open_.index).values.reshape(-1),
                         'high_fre_absdiff_vol_RC':pd.DataFrame(ff.read('high_fre_absdiff_vol_RC').loc[ff.read('synergy').index,'20200101':'20240415'],index=open_.index).values.reshape(-1),
                         'peak_count_vol_RC':pd.DataFrame(ff.read('peak_count_vol_RC').loc[ff.read('synergy').index,'20200101':'20240415'],index=open_.index).values.reshape(-1),
                         'overnightsmart20_RC':pd.DataFrame(ff.read('overnightsmart20_RC').loc[ff.read('synergy').index,'20200101':'20240415'],index=open_.index).values.reshape(-1),
                         'CTR_RC':pd.DataFrame(ff.read('CTR_RC').loc[ff.read('synergy').index,'20200101':'20240415'],index=open_.index).values.reshape(-1),
                         'jumpCTR_RC':pd.DataFrame(ff.read('jumpCTR_RC').loc[ff.read('synergy').index,'20200101':'20240415'],index=open_.index).values.reshape(-1),
                         'turnover_rate_proportion_l':pd.DataFrame(ff.read('turnover_rate_proportion_l').loc[ff.read('synergy').index,'20200101':'20240415'],index=open_.index).values.reshape(-1),
                         'synergy':pd.DataFrame(ff.read('synergy').loc[ff.read('synergy').index,'20200101':'20240415'],index=open_.index).values.reshape(-1),
                        })
stock_all
stock_code = list(np.repeat(list(ff.read('turnover_rate_proportion_l').index),len(ff.read('turnover_rate_proportion_l').loc[:,'20200101':'20240415'].columns)))
date_code = list(np.tile(list(ff.read('turnover_rate_proportion_l').loc[:,'20200101':'20240415'].columns),len(list(ff.read('turnover_rate_proportion_l').index))))
stock_all['code']= stock_code
stock_all['date']=date_code
stock_all['1'],stock_all['5'],stock_all['10'],stock_all['20'],stock_all['40'],stock_all['60']=1,5,10,20,40,60

In [ ]:
transformed_factor = []
for i, program in enumerate(test_gp._best_programs):
    print(f'Program {i+1}: {program}')
    transformed_X = program.execute(np.nan_to_num(stock_all.loc[:,fields].values))
    transformed_factor_X = pd.DataFrame(transformed_X.reshape(ff.read('accelerated_turnover_rank_RC').shape[0],-1),index = ff.read('accelerated_turnover_rank_RC').index,columns = ff.read('accelerated_turnover_rank_RC').loc[:,'20200101':'20240415'].columns)
    ff.run(-transformed_factor_X*ff.filter0.loc[:,'20200101':'20240415'], positions = 100, period = 1, fees = 0)
    ff.run(transformed_factor_X*ff.filter0.loc[:,'20200101':'20240415'], positions = 100, period = 1, fees = 0)

In [ ]:
# 截面分位数
def _rank(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        return np.nan_to_num((data_df.rank()/data_df.count()).values.reshape(-1))
    except:
        return np.zeros(len(data))

# 两组数据对应位置上的较大值
def _max(data1,data2):  
    try:
        return np.maximum(data1,data2)
    except:
        return np.zeros(len(data1))

# 两组数据对应位置上的较小值
def _min(data1,data2):  
    try:
        return np.minimum(data1,data2)
    except:
        return np.zeros(len(data1))

# 两组数据截面分位数之和
def _rank_add(data1,data2):
    try:
        data1_df=pd.DataFrame(data1.reshape(*event.shape)).replace(0,np.nan)
        data2_df=pd.DataFrame(data2.reshape(*event.shape)).replace(0,np.nan)
        rank1 = data1_df.rank()/data1_df.count()
        rank2 = data2_df.rank()/data2_df.count()
        value = rank1+rank2
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data1))

# 两组数据截面分位数之差
def _rank_sub(data1,data2):
    try:
        data1_df=pd.DataFrame(data1.reshape(*event.shape)).replace(0,np.nan)
        data2_df=pd.DataFrame(data2.reshape(*event.shape)).replace(0,np.nan)
        rank1 = data1_df.rank()/data1_df.count()
        rank2 = data2_df.rank()/data2_df.count()
        value = rank1-rank2
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data1))

# 两组数据截面分位数乘积
def _rank_mul(data1,data2):
    try:
        data1_df=pd.DataFrame(data1.reshape(*event.shape)).replace(0,np.nan)
        data2_df=pd.DataFrame(data2.reshape(*event.shape)).replace(0,np.nan)
        rank1 = data1_df.rank()/data1_df.count()
        rank2 = data2_df.rank()/data2_df.count()
        value = rank1*rank2
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data1))

# 两组数据截面分位数比值
def _rank_div(data1,data2):
    try:
        data1_df=pd.DataFrame(data1.reshape(*event.shape)).replace(0,np.nan)
        data2_df=pd.DataFrame(data2.reshape(*event.shape)).replace(0,np.nan)
        rank1 = data1_df.rank()/data1_df.count()
        rank2 = data2_df.rank()/data2_df.count()
        value = rank1/rank2
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data1))

# shift 1
def _delay1(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        return np.nan_to_num(data_df.shift(1,axis=1).values.reshape(-1))
    except:
        return np.zeros(len(data))

# shift 5
def _delay5(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        return np.nan_to_num(data_df.shift(5,axis=1).values.reshape(-1))
    except:
        return np.zeros(len(data))

# diff 1
def _delta1(data):  
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        return np.nan_to_num(data_df.diff(1,axis=1).values.reshape(-1))
    except:
        return np.zeros(len(data))

# rolling 5天取std
def _ts_std5(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value=data_df.rolling(5,min_periods=2,axis=1).std()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

# rolling 10天取std
def _ts_std10(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value=data_df.rolling(10,min_periods=5,axis=1).std()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

# rolling 5天取最小值
def _ts_min5(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value=data_df.rolling(5,min_periods=2,axis=1).min()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

# rolling 10天取最小值
def _ts_min10(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value=data_df.rolling(10,min_periods=5,axis=1).min()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

# rolling 5天取最大值
def _ts_max5(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value=data_df.rolling(5,min_periods=2,axis=1).max()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

# rolling 10天取最大值
def _ts_max10(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value=data_df.rolling(10,min_periods=5,axis=1).max()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

# rolling 10天取最小值
def _ts_cov10(data1, data2):
    try:
        data1_df=pd.DataFrame(data1.reshape(*event.shape)).replace(0,np.nan)
        data2_df=pd.DataFrame(data2.reshape(*event.shape)).replace(0,np.nan)
        value=data1_df.T.rolling(window=10, min_periods=5).cov(data2_df.T).T
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data1))

# 时序协方差
def _ts_cov5(data1, data2):
    try:
        data1_df=pd.DataFrame(data1.reshape(*event.shape)).replace(0,np.nan)
        data2_df=pd.DataFrame(data2.reshape(*event.shape)).replace(0,np.nan)
        value=data1_df.T.rolling(window=5, min_periods=2).cov(data2_df.T).T
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data1))

def _ts_corr10(data1, data2):
    try:
        data1_df=pd.DataFrame(data1.reshape(*event.shape)).replace(0,np.nan)
        data2_df=pd.DataFrame(data2.reshape(*event.shape)).replace(0,np.nan)
        value=data1_df.T.rolling(window=10, min_periods=5).corr(data2_df.T).T
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data1))

# 时序相关系数
def _ts_corr5(data1, data2):
    try:
        data1_df=pd.DataFrame(data1.reshape(*event.shape)).replace(0,np.nan)
        data2_df=pd.DataFrame(data2.reshape(*event.shape)).replace(0,np.nan)
        value=data1_df.T.rolling(window=5, min_periods=2).corr(data2_df.T).T
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data1))

# 时序 mean/std
def _ts_ms10(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        roll = data_df.rolling(window=10, min_periods=5,axis=1)
        value = roll.mean() / roll.std()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))
    
def _ts_ms5(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        roll = data_df.rolling(window=5, min_periods=2,aixs=1)
        value = roll.mean() / roll.std()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

# 时序z分数
def _ts_zscore10(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        roll = data_df.rolling(window=10, min_periods=5,axis=1)
        value = (data_df - roll.mean())/roll.std()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

def _ts_zscore5(data):
    try:
        data_df=pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        roll = data_df.rolling(window=5, min_periods=2,axis=1)
        value = (data_df - roll.mean())/roll.std()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

# 时序pct_change
def _ts_chg10(data):
    try:
        data_df = pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value = data_df.pct_change(10, axis=1)
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))
    
def _ts_chg5(data):
    try:
        data_df = pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value = data_df.pct_change(5, axis=1)
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))
    
def _ts_chg1(data):
    try:
        data_df = pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value = data_df.pct_change(1, axis=1)
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

# 时序偏度
def _ts_skew5(data):
    try:
        data_df = pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value = data_df.rolling(5,min_periods=2,axis=1).skew()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))
    
def _ts_skew10(data):
    try:
        data_df = pd.DataFrame(data.reshape(*event.shape)).replace(0,np.nan)
        value = data_df.rolling(10,min_periods=5,axis=1).skew()
        return np.nan_to_num(value.values.reshape(-1))
    except:
        return np.zeros(len(data))

rank=make_function(function=_rank,name='rank',arity=1)
max_=make_function(function=_max,name='max_',arity=2)
min_=make_function(function=_min,name='min_',arity=2)
rank_add=make_function(function=_rank_add,name='rank_add',arity=2)
rank_sub=make_function(function=_rank_sub,name='rank_sub',arity=2)
rank_mul=make_function(function=_rank_mul,name='rank_mul',arity=2)
rank_div=make_function(function=_rank_div,name='rank_div',arity=2)
delay1=make_function(function=_delay1,name='delay1',arity=1)
delay5=make_function(function=_delay5,name='delay5',arity=1)
delta1=make_function(function=_delta1,name='delta1',arity=1)
ts_std5=make_function(function=_ts_std5,name='ts_std5',arity=1)
ts_std10=make_function(function=_ts_std10,name='ts_std10',arity=1)
ts_min5=make_function(function=_ts_min5,name='ts_min5',arity=1)
ts_min10=make_function(function=_ts_min10,name='ts_min10',arity=1)
ts_max5=make_function(function=_ts_max5,name='ts_max5',arity=1)
ts_max10=make_function(function=_ts_max10,name='ts_max10',arity=1)
ts_cov10=make_function(function=_ts_cov10,name='ts_cov10',arity=2)
ts_cov5=make_function(function=_ts_cov5,name='ts_cov5',arity=2)
ts_corr10=make_function(function=_ts_corr10,name='ts_corr10',arity=2)
ts_corr5=make_function(function=_ts_corr5,name='ts_corr5',arity=2)
ts_ms10=make_function(function=_ts_ms10,name='ts_ms10',arity=1)
ts_ms5=make_function(function=_ts_ms5,name='ts_ms5',arity=1)
ts_zscore10=make_function(function=_ts_zscore10,name='ts_zscore10',arity=1)
ts_zscore5=make_function(function=_ts_zscore5,name='ts_zscore5',arity=1)
ts_chg10=make_function(function=_ts_chg10,name='ts_chg10',arity=1)
ts_chg5=make_function(function=_ts_chg5,name='ts_chg5',arity=1)
ts_chg1=make_function(function=_ts_chg1,name='ts_chg1',arity=1)
ts_skew10=make_function(function=_ts_skew10,name='ts_skew10',arity=1)
ts_skew5=make_function(function=_ts_skew5,name='ts_skew5',arity=1)

In [ ]:
function_set = ['add','sub','mul','div','sqrt','log','inv','abs','neg',rank,max_,min_,rank_add,rank_sub,rank_mul,rank_div,delay1,delay5,delta1,ts_std5,ts_std10,ts_min5,ts_min10,ts_max5,ts_max10,ts_cov10,ts_cov5,ts_corr10,ts_corr5,ts_ms10,ts_ms5,ts_zscore10,ts_zscore5,ts_chg10,ts_chg5,ts_chg1,ts_skew10,ts_skew5]

In [ ]:
test_gp=SymbolicTransformer(feature_names = fields,
                            function_set = function_set, #所有算子
                            generations = 3, #进化代数
                            population_size = 100, #种群规模
                            tournament_size = 10, #竞标赛规模
                            p_crossover=0.4,
                            p_subtree_mutation=0.05,
                            p_hoist_mutation=0.01,
                            p_point_mutation=0.03,
                            p_point_replace=0.35,
                            init_depth=(1,4),
                            const_range = None,
                            metric=rankIC,
                            parsimony_coefficient = 'auto',
                            low_memory=True,
                            verbose=2,
                            n_jobs = 6)

In [ ]:
test_gp.fit(np.nan_to_num(stock_test.loc[:,fields].values),np.nan_to_num(test_mkt))

In [ ]:
stock_all=pd.DataFrame({
    # 'open':open_.loc[:,'20200101':'20210101'].values.reshape(-1),
    #                      'close':close.loc[:,'20200101':'20210101'].values.reshape(-1),
    #                      'high':high.loc[:,'20200101':'20210101'].values.reshape(-1),
    #                      'low':low.loc[:,'20200101':'20210101'].values.reshape(-1),
    #                      'vol':vol.loc[:,'20200101':'20210101'].values.reshape(-1),
    #                      'amount':amount.loc[:,'20200101':'20210101'].values.reshape(-1),
                'accelerated_turnover_rank_RC':pd.DataFrame(ff.read('accelerated_turnover_rank_RC').loc[ff.read('synergy').index,'20200101':'20240101'],index=open_.index).values.reshape(-1),
                         'CSK_XYY_UP_DOWN_120D_RC':pd.DataFrame(ff.read('CSK_XYY_UP_DOWN_120D_RC').loc[ff.read('synergy').index,'20200101':'20240101'],index=open_.index).values.reshape(-1),
                         'high_fre_vol_RC':pd.DataFrame(ff.read('high_fre_vol_RC').loc[ff.read('synergy').index,'20200101':'20240101'],index=open_.index).values.reshape(-1),
                         'high_fre_diff_vol_RC':pd.DataFrame(ff.read('high_fre_diff_vol_RC').loc[ff.read('synergy').index,'20200101':'20240101'],index=open_.index).values.reshape(-1),
                         'high_fre_absdiff_vol_RC':pd.DataFrame(ff.read('high_fre_absdiff_vol_RC').loc[ff.read('synergy').index,'20200101':'20240101'],index=open_.index).values.reshape(-1),
                         'peak_count_vol_RC':pd.DataFrame(ff.read('peak_count_vol_RC').loc[ff.read('synergy').index,'20200101':'20240101'],index=open_.index).values.reshape(-1),
                         'overnightsmart20_RC':pd.DataFrame(ff.read('overnightsmart20_RC').loc[ff.read('synergy').index,'20200101':'20240101'],index=open_.index).values.reshape(-1),
                         'CTR_RC':pd.DataFrame(ff.read('CTR_RC').loc[ff.read('synergy').index,'20200101':'20240101'],index=open_.index).values.reshape(-1),
                         'jumpCTR_RC':pd.DataFrame(ff.read('jumpCTR_RC').loc[ff.read('synergy').index,'20200101':'20240101'],index=open_.index).values.reshape(-1),
                         'turnover_rate_proportion_l':pd.DataFrame(ff.read('turnover_rate_proportion_l').loc[ff.read('synergy').index,'20200101':'20240101'],index=open_.index).values.reshape(-1),
                         'synergy':pd.DataFrame(ff.read('synergy').loc[ff.read('synergy').index,'20200101':'20240101'],index=open_.index).values.reshape(-1),
                        })
stock_all
stock_code = list(np.repeat(list(ff.read('turnover_rate_proportion_l').index),len(ff.read('turnover_rate_proportion_l').loc[:,'20200101':'20240101'].columns)))
date_code = list(np.tile(list(ff.read('turnover_rate_proportion_l').loc[:,'20200101':'20240101'].columns),len(list(ff.read('turnover_rate_proportion_l').index))))
stock_all['code']= stock_code
stock_all['date']=date_code
stock_all['1'],stock_all['5'],stock_all['10'],stock_all['20'],stock_all['40'],stock_all['60']=1,5,10,20,40,60

In [ ]:
transformed_factor = []
for i, program in enumerate(test_gp._best_programs):
    print(f'Program {i+1}: {program}')
    transformed_X = program.execute(np.nan_to_num(stock_all.loc[:,fields].values))
    transformed_factor_X = pd.DataFrame(transformed_X.reshape(ff.read('accelerated_turnover_rank_RC').shape[0],-1),index = ff.read('accelerated_turnover_rank_RC').index,columns = ff.read('accelerated_turnover_rank_RC').loc[:,'20200101':'20240101'].columns)
    ff.run(-transformed_factor_X*ff.filter0.loc[:,'20200101':'20240101'], positions = 100, period = 1, fees = 0)
    ff.run(transformed_factor_X*ff.filter0.loc[:,'20200101':'20240101'], positions = 100, period = 1, fees = 0)